In [1]:
#Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress

covid_path = "Resources/COVID_19.csv"
GDP_path = "Resources/Global_GDP.csv"
cities_path = "Resources/worldcities.csv"

COVID_data = pd.read_csv(covid_path)
GDP_data = pd.read_csv(GDP_path)
cities_data = pd.read_csv(cities_path)

In [2]:
gdp_2019 = GDP_data[['Country Name', 'Country Code', '2019']]
gdp_2019 = gdp_2019.rename(columns={'Country Name':'Country', '2019':'2019 GDP'})
#gdp_2019.dropna(how='any')
gdp_2019.set_index('Country')

,Country Code,2019 GDP
Country,,
Aruba,ABW,NaN
Afghanistan,AFG,1.910135e+10
Angola,AGO,9.463542e+10
Albania,ALB,1.527808e+10
Andorra,AND,3.154058e+09
...,...,...
Kosovo,XKX,7.926108e+09
Yemen,YEM,NaN
South Africa,ZAF,3.510000e+11


In [3]:
COVID_data = COVID_data[['dateRep', 'cases', 'deaths', 'countriesAndTerritories', 'countryterritoryCode', 'popData2019' ]]
COVID_data = COVID_data.rename(columns={"countriesAndTerritories": "Country"})
COVID_data.set_index("Country")

covid_country_groups = COVID_data.groupby("Country")

covid_death_totals = covid_country_groups['deaths'].sum()
covid_infection_totals = covid_country_groups['cases'].sum()
covid_countries = COVID_data['Country'].unique()
country_pops = COVID_data['popData2019'].unique()
code = COVID_data['countryterritoryCode'].unique()

cleaned_covid = pd.DataFrame({
    "Cases" : covid_infection_totals,
    "Deaths" : covid_death_totals,
    "Population" : country_pops,
    "Country Code" : code
})

cleaned_covid.head()

,Cases,Deaths,Population,Country Code
Country,,,,
Afghanistan,39616,1470,38041757.0,AFG
Albania,14730,407,2862427.0,ALB
Algeria,52520,1779,43053054.0,DZA
Andorra,2568,53,76177.0,AND
Angola,5725,211,31825299.0,AGO


In [4]:

cities_data.set_index('country')
capitals = cities_data.loc[cities_data['capital'] == 'primary']
capitals = capitals[['country', 'lat', 'lng', 'iso3']]
capitals = capitals.rename(columns={'country':'Country', 'iso3':'Country Code', 'lat':'Latitude at Capital City', 'lng':'Longitude at Capital City'})
capitals

,Country,Latitude at Capital City,Longitude at Capital City,Country Code
0,Japan,35.6850,139.7514,JPN
2,Mexico,19.4424,-99.1310,MEX
9,Bangladesh,23.7231,90.4086,BGD
10,Argentina,-34.6025,-58.3975,ARG
12,Egypt,30.0500,31.2500,EGY
...,...,...,...,...
600,West Bank,31.7764,35.2269,XWB
601,Sint Maarten,18.0255,-63.0450,SXM
602,Burundi,-3.4271,29.9246,BDI
603,Kosovo,42.6666,21.1724,XKS


In [5]:
gdp_lat = pd.merge(capitals, gdp_2019, on='Country Code', how='outer')
gdp_lat

,Country_x,Latitude at Capital City,Longitude at Capital City,Country Code,Country_y,2019 GDP
0,Japan,35.6850,139.7514,JPN,Japan,5.080000e+12
1,Mexico,19.4424,-99.1310,MEX,Mexico,1.260000e+12
2,Bangladesh,23.7231,90.4086,BGD,Bangladesh,3.030000e+11
3,Argentina,-34.6025,-58.3975,ARG,Argentina,4.500000e+11
4,Egypt,30.0500,31.2500,EGY,Egypt,3.030000e+11
...,...,...,...,...,...,...
229,NaN,NaN,NaN,TCA,Turks and Caicos Islands,NaN
230,NaN,NaN,NaN,VGB,British Virgin Islands,NaN
231,NaN,NaN,NaN,VIR,Virgin Islands (U.S.),NaN
232,NaN,NaN,NaN,WLD,World,8.770000e+13


In [6]:
merged = pd.merge(gdp_lat, cleaned_covid, on='Country Code', how='outer')
merged

,Country_x,Latitude at Capital City,Longitude at Capital City,Country Code,Country_y,2019 GDP,Cases,Deaths,Population
0,Japan,35.6850,139.7514,JPN,Japan,5.080000e+12,87020.0,1613.0,126860299.0
1,Mexico,19.4424,-99.1310,MEX,Mexico,1.260000e+12,774020.0,80083.0,127575529.0
2,Bangladesh,23.7231,90.4086,BGD,Bangladesh,3.030000e+11,373151.0,5440.0,163046173.0
3,Argentina,-34.6025,-58.3975,ARG,Argentina,4.500000e+11,840902.0,22226.0,44780675.0
4,Egypt,30.0500,31.2500,EGY,Egypt,3.030000e+11,104035.0,6010.0,100388076.0
...,...,...,...,...,...,...,...,...,...
239,NaN,NaN,NaN,VAT,NaN,NaN,12.0,0.0,815.0
240,NaN,NaN,NaN,JEY,NaN,NaN,441.0,32.0,107796.0
241,NaN,NaN,NaN,MSF,NaN,NaN,13.0,1.0,4991.0
242,NaN,NaN,NaN,CNG1925,NaN,NaN,524.0,7.0,23773881.0


In [7]:
merged['Country Code'].unique()

array(['JPN', 'MEX', 'BGD', 'ARG', 'EGY', 'CHN', 'PHL', 'RUS', 'FRA',
       'KOR', 'IDN', 'GBR', 'PER', 'IRN', 'COD', 'COL', 'TWN', 'THA',
       'CHL', 'ESP', 'USA', 'SGP', 'AGO', 'IRQ', 'SDN', 'SAU', 'VNM',
       'MMR', 'CIV', 'BRA', 'TUR', 'DEU', 'DZA', 'ITA', 'PRK', 'AFG',
       'GRC', 'ZAF', 'ETH', 'KEN', 'VEN', 'TZA', 'PRT', 'UKR', 'SEN',
       'SYR', 'TUN', 'AUT', 'LBY', 'UZB', 'CUB', 'DOM', 'AZE', 'GHA',
       'KWT', 'YEM', 'HTI', 'ROU', 'PRY', 'LBN', 'BLR', 'BEL', 'POL',
       'MAR', 'ECU', 'MDG', 'HUN', 'CMR', 'BOL', 'NGA', 'ZWE', 'URY',
       'MLI', 'GIN', 'KHM', 'TGO', 'QAT', 'MYS', 'MOZ', 'SLV', 'UGA',
       'NLD', 'COG', 'ZMB', 'CRI', 'PAN', 'SWE', 'BGR', 'CZE', 'BFA',
       'CAN', 'FIN', 'ARM', 'SOM', 'GEO', 'SRB', 'TJK', 'DNK', 'JOR',
       'IRL', 'LBR', 'ISR', 'GTM', 'TCD', 'HND', 'JAM', 'DJI', 'NIC',
       'NER', 'ALB', 'NPL', 'MNG', 'RWA', 'KGZ', 'NOR', 'CAF', 'SLE',
       'PAK', 'BEN', 'LAO', 'LVA', 'MRT', 'OMN', 'TKM', 'HRV', 'BIH',
       'MDA', 'MWI',

In [8]:
capitals['Country'].unique()

array(['Japan', 'Mexico', 'Bangladesh', 'Argentina', 'Egypt', 'China',
       'Philippines', 'Russia', 'France', 'Korea, South', 'Indonesia',
       'United Kingdom', 'Peru', 'Iran', 'Congo (Kinshasa)', 'Colombia',
       'Taiwan', 'Thailand', 'Chile', 'Spain', 'United States',
       'Singapore', 'Angola', 'Iraq', 'Sudan', 'Saudi Arabia', 'Vietnam',
       'Burma', 'Côte D’Ivoire', 'Brazil', 'Turkey', 'Germany', 'Algeria',
       'Italy', 'Korea, North', 'Afghanistan', 'Greece', 'South Africa',
       'Ethiopia', 'Kenya', 'Venezuela', 'Tanzania', 'Portugal',
       'Ukraine', 'Senegal', 'Syria', 'Tunisia', 'Austria', 'Libya',
       'Uzbekistan', 'Cuba', 'Dominican Republic', 'Azerbaijan', 'Ghana',
       'Kuwait', 'Yemen', 'Haiti', 'Romania', 'Paraguay', 'Lebanon',
       'Belarus', 'Belgium', 'Poland', 'Morocco', 'Ecuador', 'Madagascar',
       'Hungary', 'Cameroon', 'Bolivia', 'Nigeria', 'Zimbabwe', 'Uruguay',
       'Mali', 'Guinea', 'Cambodia', 'Togo', 'Qatar', 'Malaysia',
      

In [9]:
#cleaned_covid['Country'].unique()
covid_countries

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antigua_and_Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia',
       'Bonaire, Saint Eustatius and Saba', 'Bosnia_and_Herzegovina',
       'Botswana', 'Brazil', 'British_Virgin_Islands',
       'Brunei_Darussalam', 'Bulgaria', 'Burkina_Faso', 'Burundi',
       'Cambodia', 'Cameroon', 'Canada', 'Cape_Verde',
       'Cases_on_an_international_conveyance_Japan', 'Cayman_Islands',
       'Central_African_Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo', 'Costa_Rica', 'Cote_dIvoire', 'Croatia',
       'Cuba', 'Curaçao', 'Cyprus', 'Czechia',
       'Democratic_Republic_of_the_Congo', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican_Republic', 'Ecuador', 'Egypt',
       'El_Salvador', 'Equatorial_Guinea', 'Eri

In [10]:
gdp_2019['Country'].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Albania', 'Andorra',
       'United Arab Emirates', 'Argentina', 'Armenia', 'American Samoa',
       'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan',
       'Burundi', 'Belgium', 'Benin', 'Burkina Faso', 'Bangladesh',
       'Bulgaria', 'Bahrain', 'Bahamas', 'Bosnia and Herzegovina',
       'Belarus', 'Belize', 'Bermuda', 'Bolivia', 'Brazil', 'Barbados',
       'Brunei Darussalam', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland',
       'Channel Islands', 'Chile', 'China', "Cote d'Ivoire", 'Cameroon',
       'Congo, Dem. Rep.', 'Congo, Rep.', 'Colombia', 'Comoros',
       'Cabo Verde', 'Costa Rica', 'Cuba', 'Curacao', 'Cayman Islands',
       'Cyprus', 'Czech Republic', 'Germany', 'Djibouti', 'Dominica',
       'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador', 'Egypt',
       'Eritrea', 'Spain', 'Estonia', 'Ethiopia', 'European Union',
       'Finland', 'Fiji', 'France', 'Faroe Islands', 'Micron